In [83]:
from metaflow import Flow, Step

flow = Flow("CoOcurrenceMatrixFlow")

In [84]:
flow.latest_run.id


'1678617048838446'

In [85]:
list(flow.latest_run.steps())

[Step('CoOcurrenceMatrixFlow/1678617048838446/end'),
 Step('CoOcurrenceMatrixFlow/1678617048838446/build_cooccurence_matrix'),
 Step('CoOcurrenceMatrixFlow/1678617048838446/build_reference_dicts'),
 Step('CoOcurrenceMatrixFlow/1678617048838446/start')]

In [86]:
co_occurrence_matrix = Step(f"CoOcurrenceMatrixFlow/{flow.latest_run.id}/end").task.data.matrix.astype(float)


In [87]:
co_occurrence_matrix

<12423x12423 sparse matrix of type '<class 'numpy.float64'>'
	with 15591985 stored elements in COOrdinate format>

In [90]:
from scipy.sparse.linalg import svds
import numpy as np

In [106]:
U, S, Vt = svds(co_occurrence_matrix, k=3)

In [107]:
embeddings = U * np.sqrt(S)

# Normalize embeddings
embeddings = (embeddings - np.mean(embeddings, axis=0)) / np.std(embeddings, axis=0)

In [108]:
embeddings

array([[0.13566082, 0.03562961, 0.15710274],
       [0.12194674, 0.044861  , 0.16135526],
       [0.11104908, 0.00411357, 0.11798021],
       ...,
       [0.16357064, 0.05841535, 0.12666839],
       [0.0619011 , 0.04355717, 0.15609396],
       [0.13326719, 0.03691926, 0.16281501]])

In [104]:
import tensorflow as tf
from tensorboard.plugins import projector
import numpy as np

# Set up a TensorFlow summary writer
log_dir = 'logs/embeddings'
summary_writer = tf.summary.create_file_writer(log_dir)

# Create the embedding variable
embedding_var = tf.Variable(embeddings, name='embedding')

# Configure the projector
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Write the metadata to a file
metadata_path = f'{log_dir}/metadata.tsv'
with open(metadata_path, 'w') as metadata_file:
    metadata_file.write('\n'.join(['label_{}'.format(i) for i in range(len(embeddings))]))

embedding.metadata_path = metadata_path

# Write the projector to disk
with summary_writer.as_default():
    tf.summary.scalar('my_scalar', 0.5, step=0)
    tf.summary.tensor_embedding(config)

# Close the summary writer
summary_writer.close()
summary_writer.close()

AttributeError: module 'tensorboard.summary._tf.summary' has no attribute 'tensor_embedding'